In [ ]:
import sys
import os
#sys.path.append(os.path.abspath('.'))
import csv
from lista_pregoes import *
from obter_url_dos_itens import *
from atualizar_todos_os_dados import *
from juntar_df import *
from atualizar_saldo_vigencia import *

In [ ]:
try:    
    url_pregoes_antiga = carregar_lista_pregoes()

except:    
    print('Não há Lista de Pregões salva no computador.')
    url_pregoes_antiga = []

url_pregoes_nova = baixar_nova_lista_pregoes()

#Verificando se há novos pregões
    
novos_pregoes = list(set(url_pregoes_nova) - set(url_pregoes_antiga))
novos_pregoes = sorted(novos_pregoes)

print(f'Há {len(novos_pregoes)} novos pregões!')


In [ ]:
if not novos_pregoes:
    
    #df_itens_novos=None
    
    print('Não há novos pregões.')

    #atualizando saldo e vigência
    df_itens_final = atualizar_saldo_vigencia('df_itens_participante.csv')

    #Salvando o df em arquivo csv
    nome_arquivo = "df_itens_participante.csv"
    df_itens_final.sort_values(['Número da Compra'], ascending = [False], inplace = True)
    df_itens_final.to_csv(nome_arquivo, sep=';', encoding='utf-8', index=False)        
    print('Arquivo atualizado com sucesso!')
            
else:

    try:

        df_itens_final = pd.read_csv('df_itens_participante.csv', sep = ';')
        print('Tabela de Dados encontrado no computador!')

        #atualizando saldo e vigência
        df_itens_final = atualizar_saldo('df_itens_participante.csv')

    except:

        df_itens_final = pd.DataFrame(columns = ['Número da Compra', 'Número do Item', 'Descrição',
       'Descrição Detalhada', 'Início da Vigência', 'Fim da Vigência',
       'Unidade', 'Qtd. Autorizada', 'Fornecedor', 'Val. Unitário',
       'Qtd. Saldo', 'Link do Item', 'Tipo de Compra', 'Número do Pregão',
       'Ano do Pregão', 'UASG', 'Objeto', 'Unidade de Fornecimento'])
    
    for url in novos_pregoes:
        
        #obter a url de cada item do pregão
        url_itens_novos = obter_url_dos_itens([url])

        #obter os dado de cada item 
        df_itens_novos = obter_dados_site_novo(url_itens_novos)
        print('Dados dos itens dos novos pregões obtidos com sucesso!')  
    
        df_itens_novos_site_antigo = obter_unidade_objeto(df_itens_novos)
        print('Objeto e Unidade de Fornecimento dos novos pregões obtidos com sucesso!') 
        
        df_itens_merge = merge_df(df_itens_novos,df_itens_novos_site_antigo)
        print('Dados mesclados com sucesso!')

        df_itens_final = pd.concat([df_itens_final, df_itens_merge])
        print('Tabela dos novos itens adicionada com sucesso!')

        #Salvando o df em arquivo csv
        nome_arquivo = "df_itens_participante.csv"
        df_itens_final.sort_values(['Número da Compra'], ascending = [False], inplace = True)
        df_itens_final.to_csv(nome_arquivo, sep=';', encoding='utf-8', index=False)        
        print('Arquivo com a tabela dos itens salva com sucesso!')

        #guardando o link dos pregões que já tiveram os dados obtidos
        url_pregoes_antiga.append(url)
        
        # Gravar a lista em um arquivo CSV        
        with open('url_pregoes.csv', mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows([[url] for url in url_pregoes_antiga])
            print(f'Dados do Pregão da url {url} obtidos com sucesso! \n')